In [158]:
import sympy as sp
import numpy as np

In [1]:
from partitions import *
from characters import *
from complete import *
from schur import *

# Special Cases

## Special Case 01:

In [13]:
z = symbols('z')
t[0] = z
for n in range(1,len(t)):
    t[n] = z**(n+1) / (n+1)

In [14]:
t

array([z, z**2/2, z**3/3, z**4/4, z**5/5, z**6/6, z**7/7, z**8/8, z**9/9,
       z**10/10, z**11/11, z**12/12], dtype=object)

In [31]:
par1 = [6,3,2]
par2 = [5,3,0]
interlaced(par1, par2)

True

In [32]:
skewSchur(par1, par2)

z**3

## Special Case 02:

In [33]:
z = symbols('z')
t[0] = z
for n in range(1,len(t)):
    #t[n] = z**(n+1) / (n+1)
    t[n] = 0

In [34]:
t

array([z, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=object)

In [38]:
par1 = [3,2,1]
par2 = [2,1,0]
interlaced(par1, par2)

True

In [39]:
skewSchur(par1, par2)

z**3

# Ground State

In [40]:
from sympy.physics.quantum import Ket

In [41]:
Y1, Y2, Y3, Y4 = symbols('Y1, Y2, Y3, Y4') # These are auxiliary coordinates

In [42]:
def states(lev):
    #Lamb = []
    partitions = bosonic_states_partition(lev)
    vectors_k = bosonic_states(lev)
    num_part = len(partitions)
    summ = 0
    
    for (n,m) in product(range(num_part), repeat=2):
        #print(partitions[m])
       
        z = 1
        for index in range(lev):
            z *= factorial(vectors_k[n][index]) * (index + 1)**vectors_k[n][index] 

        power = Y1**(partitions[m][0]) * Y2**(partitions[m][1]) * Y3**(partitions[m][2]) * Y4**(partitions[m][3])
        state = Ket(partitions[m][0], partitions[m][1], partitions[m][2])#, partitions[m][3])

        summ += sqrt(Rational(1, z)) * Characters(vectors_k[n], partitions[m]) * power * state
        
    return summ

In [43]:
S = states(4)

In [44]:
S

sqrt(6)*Y1**4*|400>/12 + sqrt(2)*Y1**4*|400>/4 + sqrt(3)*Y1**4*|400>/3 + Y1**4*|400> - sqrt(2)*Y1**3*Y2*|310>/4 + sqrt(6)*Y1**3*Y2*|310>/4 - sqrt(3)*Y1**2*Y2**2*|220>/3 + sqrt(6)*Y1**2*Y2**2*|220>/6 + sqrt(2)*Y1**2*Y2**2*|220>/2 - sqrt(2)*Y1**2*Y2*Y3*|211>/4 + sqrt(6)*Y1**2*Y2*Y3*|211>/4 - Y1*Y2*Y3*Y4*|111> + sqrt(6)*Y1*Y2*Y3*Y4*|111>/12 + sqrt(2)*Y1*Y2*Y3*Y4*|111>/4 + sqrt(3)*Y1*Y2*Y3*Y4*|111>/3

In [45]:
a = S.diff(Y1,4)/ factorial(4)
a

(sqrt(6)*|400> + 3*sqrt(2)*|400> + 4*sqrt(3)*|400> + 12*|400>)/12

In [46]:
b = S.diff(Y1,3).diff(Y2)/ factorial(3).simplify()
b 

(-3*sqrt(2)*|310> + 3*sqrt(6)*|310>)/12

In [47]:
c = S.diff(Y1,2).diff(Y2,2)/4
c

(-4*sqrt(3)*|220>/3 + 2*(sqrt(6)*|220>)/3 + 2*(sqrt(2)*|220>))/4

In [48]:
d = S.diff(Y1,2).diff(Y2).diff(Y3)/2
d

(-3*sqrt(2)*|211> + 3*sqrt(6)*|211>)/12

In [49]:
e = S.diff(Y1).diff(Y2).diff(Y3).diff(Y4)
e

-|111> + sqrt(6)*|111>/12 + sqrt(2)*|111>/4 + sqrt(3)*|111>/3

In [50]:
(a**2 + b**2 + c**2 + d**2 + e**2).simplify()

(18*(-|211> + sqrt(3)*|211>)**2 + 18*(-|310> + sqrt(3)*|310>)**2 + 4*(-2*sqrt(3)*|220> + sqrt(6)*|220> + 3*sqrt(2)*|220>)**2 + (sqrt(6)*|400> + 3*sqrt(2)*|400> + 4*sqrt(3)*|400> + 12*|400>)**2 + (-12*|111> + sqrt(6)*|111> + 3*sqrt(2)*|111> + 4*sqrt(3)*|111>)**2)/144

# AntiCommutation Relations

In [92]:
m, n, i, j, k, Q, L = sp.symbols('m, n, i, j, k, Q, L')

In [93]:
psi = sp.Function('psi', commutative=False)
psi_star = sp.Function('psi^*', commutative=False)

In [146]:
def PP(x, y):
    return Q * ( psi(x + 1) * psi(y-1) +  psi(y + 1) * psi(x -1) )

def PstarPstar(x, y):
    return Q * ( psi_star(x - 1) * psi_star(y + 1) +  psi_star(y - 1) * psi_star(x+1) )

def PPstar(x, y):
    return Q * ( psi(x - 1) * psi_star(y-1) +  psi_star(y + 1) * psi(x + 1) ) + (1 - Q)**2 * sp.KroneckerDelta(x,y)

In [186]:
PPstar(-3,-2)

Q*(psi(-4)*psi^*(-3) + psi^*(-1)*psi(-2))

In [189]:
PstarPstar(-1,-1)

2*Q*psi^*(-2)*psi^*(0)

In [142]:
def exp_PstarP(x, y):
    return Q * psi(y -1) * psi_star(x -1) + (Q**2 - 1) * sp.Sum(Q**j * psi(y + j) * psi_star(x + j), (j,0, L)) + (1-Q) * sp.KroneckerDelta(x,y)

def exp_PPstar(x, y):
    return Q * psi_star(y + 1) * psi(x + 1) + (Q**2 - 1) * sp.Sum(Q**j * psi_star(y - j) * psi(x - j), (j,0, L))+ (1-Q) * sp.KroneckerDelta(x,y)

def exp_PP(x, y):
    return psi(y) * psi(x) + Q * ( psi(x + 1) * psi(y - 1) + psi(y + 1) * psi(x - 1))

def anticom_PstarPstar(x, y):
    return psi_star(y) * psi_star(x) + Q * ( psi_star(x - 1) * psi_star(y + 1) + psi_star(y - 1) * psi_star(x + 1))

In [150]:
exp_PPstar(-2, -2)

-Q + Q*psi^*(-1)*psi(-1) + (Q**2 - 1)*Sum(Q**j*psi^*(-j - 2)*psi(-j - 2), (j, 0, L)) + 1

In [151]:
exp_PstarP(1, 1)

-Q + Q*psi(0)*psi^*(0) + (Q**2 - 1)*Sum(Q**j*psi(j + 1)*psi^*(j + 1), (j, 0, L)) + 1

In [159]:
x  = np.array(sp.symbols('x1:{}'.format(3)))

In [160]:
x**2

array([x1**2, x2**2], dtype=object)

In [163]:
t = [ sum(x**j)/j for j in range(1,6)]

In [173]:
t[1]

x1**2/2 + x2**2/2

In [183]:
((1 + Q) * t[1] + ((1 - Q) / 2 )* t[0]**2 ).expand().simplify()

-Q*x1*x2 + x1**2 + x1*x2 + x2**2

# Other Things

In [5]:
x = (1,2,3,4,5,6)

In [8]:
A = all([x[i] <= x[i+1] for i in range(len(x)-1)])

In [9]:
A

True

In [10]:
type(x)

tuple

In [11]:
x.sort()

AttributeError: 'tuple' object has no attribute 'sort'

In [12]:
x = tuple(x)

In [13]:
x

(1, 2, 3, 4, 5, 6)

In [15]:
type(x)

tuple

In [ ]:
@DataClass